<a href="https://colab.research.google.com/github/SamTremblay18/Hockey-Shot-Classification/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries 

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
from google.colab import drive 
drive.mount('/content/drive')
import pickle
from sklearn.preprocessing import LabelEncoder, MaxAbsScaler, StandardScaler
import matplotlib.pyplot as plt
from scipy import signal
from sklearn.model_selection import train_test_split
from itertools import chain

Mounted at /content/drive


# Functions

Save and open files with pickle

In [2]:
def save_pickle(path_pickle,P):
  f = open(path_pickle, 'wb')
  pickle.dump(P, f)
  f.close()

  return f

def open_pickle(path_pickle):
  f = open(path_pickle, 'rb')
  P = pickle.load(f)
  f.close()

  return P  

Regroups all the Pre, SH and Post trials together to the label 'Other'

In [3]:
def regroup_other(P_markers,P_acc):
  for i in range(0,len(P_markers.iloc[:,0])):
    
    length_trial = P_markers.iloc[i,2] 
    start_trial = P_markers.iloc[i,0]  
   
    if P_markers.iloc[i,1] == 'Pre' or P_markers.iloc[i,1] == 'SH' or P_markers.iloc[i,1] == 'post':
      P_acc.loc[(start_trial):(start_trial)+((length_trial)-1),'Label'] = 'Other' # Swapping the label to Other

  return P_acc

Reframe all the shooting trials the same frame length

In [4]:
def reframe(P_markers,P_acc,f_length):
  for i in range(0,len(P_markers.iloc[:,0])):

    length_trial = P_markers.iloc[i,2]
    start_trial = P_markers.iloc[i,0]
    type_shot = P_markers.iloc[i,1]

    if P_markers.iloc[i,1] == 'BH' or P_markers.iloc[i,1] == 'Pass' or P_markers.iloc[i,1] == 'SS' or P_markers.iloc[i,1] == 'WS' or P_markers.iloc[i,1] == 'OT':
        diff = int(f_length-length_trial) # Identifying how many frames are missing to have the desired length
        P_acc.iloc[int(start_trial)-diff:int(start_trial)+int(length_trial),-1] = type_shot # Swapping the old labels from the difference 
                                                                                            # found above to the new label desired
  return P_acc

Standardize and label encoder function

In [7]:
def stand_labelencoder(P_acc):
  scaler = StandardScaler()
  le = LabelEncoder()
  scaled = scaler.fit_transform(P_acc.iloc[:,1:-1], y = P_acc['Label'])
  y = le.fit_transform(P_acc['Label'])
  classes = le.classes_
  y = y[:,None]
  x = np.concatenate((scaled, y), axis=1)

  return x, classes

Create tensors of specific shape

In [57]:
def create_tensor(P_acc,f_length):
  trials_length = len(P_acc)/f_length
  T = np.stack(np.split(P_acc, trials_length), axis=1)
  Tensors = tf.constant(T.astype('float32'))
  Tensors = tf.transpose(Tensors, perm = [1, 0, 2])

  return Tensors

Create 15 trials of 'Other' with specific length

In [9]:
# The majority of the data is in the category 'Other', so this function randomly creates
# 15 trials from data labeled as 'Other' with the specified length
def other_class(P_other,f_length):
  random.seed(4)
  num_list = random.sample(range(0,len(P_other),f_length),15) # this creates 15 numbers that are seperated with at least
                                                              # the specified length from all the data labeled as 'Other'
  Other_all = pd.DataFrame()
  for i in num_list:
    other = P_other.iloc[i:(i+f_length),:] # creates the trials with the specified length starting with number from the list
    Other_all = pd.concat([Other_all, other], ignore_index = True)

  return Other_all

Runs all the preprocessing steps for Method 1

In [10]:
def Preprocess_1(P_markers, P_acc):
  P_data = regroup_other(P_markers,P_acc) # regroups the 'SH', 'Pre', 'post' trials to the label 'Other'
  P_data = reframe(P_markers,P_data,576) # reframe the dataset to 576 frames
  P_other = P_data[(P_data['Label'] == 'Other')] # creates a variable containing all the data labeled 'Other'
  P_other_all = other_class(P_other,576) # from the variable above, creates 15 trials of length 576
  P_data = P_data.drop(P_data[(P_data['Label'] == 'Do not use')].index) # removing all the data labeled as 'Do not use' (junk)
  P_data = P_data.drop(P_data[(P_data['Label'] == 'Other')].index) # removing all the data labeled as 'Other' from the original dataset
  P_data = pd.concat([P_data, P_other_all], ignore_index = True) # adding the 15 'Other' trials of length 576 to the original dataset 
  P_data, classes = stand_labelencoder(P_data)
  P_data = create_tensor(P_data,576) # transforming the dataset into tensors with a length of 576 frames
  
  return P_data, classes

Runs all the preprocessing steps for Method 2

In [11]:
# This function is the same as for method 1, but the biomechanics sensor configuration is selected at the start
def Preprocess_2(P_markers, P_acc):
  P_data = P_acc.loc[:,('Frame','Right Hand x','Right Hand y','Right Hand z','Left Hand x','Left Hand y','Left Hand z',
                  'Right Foot x','Right Foot y','Right Foot z','Left Foot x','Left Foot y','Left Foot z',
                  'Pelvis x','Pelvis y','Pelvis z','Label')]
  P_data = regroup_other(P_markers,P_data)
  P_data = reframe(P_markers,P_data,576)
  P_other = P_data[(P_data['Label'] == 'Other')]
  P_other_all = other_class(P_other,576)
  P_data = P_data.drop(P_data[(P_data['Label'] == 'Do not use')].index)
  P_data = P_data.drop(P_data[(P_data['Label'] == 'Other')].index)
  P_data = pd.concat([P_data, P_other_all], ignore_index = True)
  P_data, classes = stand_labelencoder(P_data)
  P_data = create_tensor(P_data,576)
  
  return P_data, classes

Resample the shot trials to a set frequency

In [13]:
def resampling_data(markers_number, p_resample, p_acc, sampling_freq):

  for i in range(0,len(markers[markers_number].iloc[:-1,2])):

          trial_start = markers[markers_number].iloc[i,0]
          typeshot = markers[markers_number].iloc[i,1]
          trial_len = markers[markers_number].iloc[i,2]

          if typeshot == 'WS':
            # Resamples a trial by selecting all the data except the 'Frame' and 'label' column to a specified frequency
            data_resample[p_resample][f'WS_{i}'] = signal.resample(data_biomech_config[p_acc].iloc[int(trial_start):(int(trial_start)+int(trial_len)),1:-1],sampling_freq)
            # Adds the labels encoded manually at the last column position
            data_resample[p_resample][f'WS_{i}'] = np.append(data_resample[p_resample][f'WS_{i}'], np.full((sampling_freq,1), 6), axis=1)

          elif typeshot == 'BH':
            data_resample[p_resample][f'BH_{i}'] = signal.resample(data_biomech_config[p_acc].iloc[int(trial_start):(int(trial_start)+int(trial_len)),1:-1],sampling_freq)
            data_resample[p_resample][f'BH_{i}'] = np.append(data_resample[p_resample][f'BH_{i}'], np.full((sampling_freq,1), 0), axis=1)

          elif typeshot == 'OT':
            data_resample[p_resample][f'OT_{i}'] = signal.resample(data_biomech_config[p_acc].iloc[int(trial_start):(int(trial_start)+int(trial_len)),1:-1],sampling_freq)
            data_resample[p_resample][f'OT_{i}'] = np.append(data_resample[p_resample][f'OT_{i}'], np.full((sampling_freq,1), 1), axis=1)

          elif typeshot == 'SS':
            data_resample[p_resample][f'SS_{i}'] = signal.resample(data_biomech_config[p_acc].iloc[int(trial_start):(int(trial_start)+int(trial_len)),1:-1],sampling_freq)
            data_resample[p_resample][f'SS_{i}'] = np.append(data_resample[p_resample][f'SS_{i}'], np.full((sampling_freq,1), 5), axis=1)

          elif typeshot == 'Pass':
            data_resample[p_resample][f'Pass_{i}'] = signal.resample(data_biomech_config[p_acc].iloc[int(trial_start):(int(trial_start)+int(trial_len)),1:-1],sampling_freq)
            data_resample[p_resample][f'Pass_{i}'] = np.append(data_resample[p_resample][f'Pass_{i}'], np.full((sampling_freq,1), 3), axis=1)

          else:
            continue

#Method 1 - Reframe to 576 frames and All Sensors Configuration

In [119]:
# Participants' and markers' dictionnary
data = open_pickle('/content/drive/MyDrive/Sam/File_saved/P_dct.pickle')
markers = open_pickle('/content/drive/MyDrive/Sam/File_saved/markers_dct.pickle')

In [58]:
# Preprocess function for method 1 and storing into P_processed1 dictionary
P_processed1 = {}
for l,n in zip(markers, data):
  P_processed1[f'{n}_processed'], data_classes1 = Preprocess_1(markers[l],data[n])

In [59]:
data_classes1

array(['BH', 'OT', 'Other', 'Pass', 'Rest', 'SS', 'WS'], dtype=object)

In [79]:
# Splitting participants' tensor into data and labels
tensorvalues1 = list(P_processed1.values())
all_data1 = tf.concat(tensorvalues1, axis = 0)
X1 = all_data1[:,:,0:51].numpy()
y1 = all_data1[:,0,51].numpy()

In [80]:
# Train/test split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

print("X_train:",X_train1.shape,
      "X_test:",X_test1.shape,
      "y_train:",y_train1.shape,
      "y_test:",y_test1.shape)

X_train: (2655, 576, 51) X_test: (664, 576, 51) y_train: (2655,) y_test: (664,)


In [78]:
save_pickle('/content/drive/MyDrive/Sam/File_saved/X_test1.pickle',X_test1)
save_pickle('/content/drive/MyDrive/Sam/File_saved/X_train1.pickle',X_train1)
save_pickle('/content/drive/MyDrive/Sam/File_saved/y_test1.pickle',y_test1)
save_pickle('/content/drive/MyDrive/Sam/File_saved/y_train1.pickle',y_train1)

<_io.BufferedWriter name='/content/drive/MyDrive/Sam/File_saved/y_train1.pickle'>

# Method 2 - Reframe to 576 frames and Biomechanics Sensor Configuration (BSC)
 

In [ ]:
data = open_pickle('/content/drive/MyDrive/Sam/File_saved/P_dct.pickle')
markers = open_pickle('/content/drive/MyDrive/Sam/File_saved/markers_dct.pickle')

In [81]:
# Same method as above, but only the hands, foot and pelvis sensors are selected (BSC)
# Preprocess function for method 2 and storing into 'P_processed2' dictionary
P_processed2 = {}
for l,n in zip(markers, data):
  P_processed2[f'{n}_processed2'], data_classes2 = Preprocess_2(markers[l],data[n])

In [137]:
data_classes2

array(['BH', 'OT', 'Other', 'Pass', 'Rest', 'SS', 'WS'], dtype=object)

In [83]:
# Splitting participants' tensor into data and labels
tensorvalues2 = list(P_processed2.values())
all_data2 = tf.concat(tensorvalues2, axis = 0)
X2 = all_data2[:,:,0:15].numpy()
y2 = all_data2[:,0,15].numpy()

In [84]:
# Train/test split
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)

print("X_train:",X_train2.shape,
      "X_test:",X_test2.shape,
      "y_train:",y_train2.shape,
      "y_test:",y_test2.shape)

X_train: (2655, 576, 15) X_test: (664, 576, 15) y_train: (2655,) y_test: (664,)


In [85]:
save_pickle('/content/drive/MyDrive/Sam/File_saved/X_test2.pickle',X_test2)
save_pickle('/content/drive/MyDrive/Sam/File_saved/X_train2.pickle',X_train2)
save_pickle('/content/drive/MyDrive/Sam/File_saved/y_test2.pickle',y_test2)
save_pickle('/content/drive/MyDrive/Sam/File_saved/y_train2.pickle',y_train2)

<_io.BufferedWriter name='/content/drive/MyDrive/Sam/File_saved/y_train2.pickle'>

# Method 3 - Resampling and BSC

In [14]:
data = open_pickle('/content/drive/MyDrive/Sam/File_saved/P_dct.pickle')
markers = open_pickle('/content/drive/MyDrive/Sam/File_saved/markers_dct.pickle')

In [15]:
# Empty dictionary for place holding
data_resample = {'P01_resample_bmech':{},'P02_resample_bmech':{},'P03_resample_bmech':{},'P04_resample_bmech':{},'P05_resample_bmech':{},'P07_resample_bmech':{},'P08_resample_bmech':{},
                 'P09_resample_bmech':{},'P10_resample_bmech':{},'P11_resample_bmech':{},'P12_resample_bmech':{},'P13_resample_bmech':{},'P14_resample_bmech':{},'P15_resample_bmech':{},
                 'P16_resample_bmech':{},'P17_resample_bmech':{},'P18_resample_bmech':{},'P19_resample_bmech':{},'P20_resample_bmech':{},'P21_resample_bmech':{},'P22_resample_bmech':{},
                 'P23_resample_bmech':{},'P24_resample_bmech':{},'P25_resample_bmech':{},'P26_resample_bmech':{},'P27_resample_bmech':{},'P28_resample_bmech':{},'P29_resample_bmech':{},
                 'P32_resample_bmech':{},'P33_resample_bmech':{},'P34_resample_bmech':{},'P35_resample_bmech':{},'P36_resample_bmech':{},'P37_resample_bmech':{},'P39_resample_bmech':{},
                 'P40_resample_bmech':{},'P41_resample_bmech':{},'P42_resample_bmech':{},'P43_resample_bmech':{}}

In [16]:
# Stores the data with the BSC
data_biomech_config = {}
for i in data:
  data_biomech_config[f'{i}_bmech'] = data[i].loc[:,('Frame','Right Hand x','Right Hand y','Right Hand z','Left Hand x','Left Hand y','Left Hand z',
                                               'Right Foot x','Right Foot y','Right Foot z','Left Foot x','Left Foot y','Left Foot z',
                                               'Pelvis x','Pelvis y','Pelvis z','Label')]

In [17]:
# Runs some preprocessing steps and storing it into a new dictionary 'data_biomech'
data_biomech = {}
for j1,j2 in zip(markers, data_biomech_config):

  P_data = regroup_other(markers[j1], data_biomech_config[j2]) # regroups the 'SH', 'Pre', 'post' trials to the label 'Other'
  P_other = P_data[(P_data['Label'] == 'Other')] # creates a variable containing all the data labeled 'Other'
  P_other_all = other_class(P_other,400) # from the variable above, creates 15 trials of length 400
  P_data = P_data.drop(P_data[(P_data['Label'] == 'Do not use')].index) # removing all the data labeled as 'Do not use' (junk)
  P_data = P_data.drop(P_data[(P_data['Label'] == 'Other')].index) # removing all the data labeled as 'Other' from the original dataset
  P_data = pd.concat([P_data, P_other_all], ignore_index = True) # adding the 15 'Other' trials of length 400 to the original dataset
  #P_data = P_stand_norm(P_data) # standardize and normalize the dataset
  #data_biomech[f'{j2}'], data_classes3 = label_encoder(P_data) # encoding the labels
  data_biomech[f'{j2}'], data_classes3 = stand_labelencoder(P_data)

In [ ]:
data_classes3

array(['BH', 'OT', 'Other', 'Pass', 'Rest', 'SS', 'WS'], dtype=object)

In [18]:
# This runs 3 different functions from the same 3 dictionaries
data_other = {} 
data_rest = {}
for m1,m2,m3 in zip(markers, data_resample, data_biomech):
  # Resampling the shooting trials to 400 frames and store them into the 'data_resample' dictionary
  resampling_data(m1,m2,m3, 400)
  # Creates a tensor of 'Other' from the 'data_biomech' dictionary and swap the axes to the required shape
  other = data_biomech[m3][-6000:,:]
  other = create_tensor(other, 400)
  data_other[f'{m3}_other'] = tf.transpose(other, perm = [1,0,2])
  # Creates a tensor of 'Rest' from the data_biomech' dictionary and swap the axes to the required shape
  rest = data_biomech[m3][(data_biomech[m3][:,-1] == 4)]
  rest = rest[0:6000,:]
  rest = create_tensor(rest,400)
  data_rest[f'{m3}_rest'] = tf.transpose(rest, perm = [1, 0, 2])

In [19]:
# Converting the shots data from the 'data_resample' dictionary into tensor and store it into 'data_tensor' dictionary
data_tensor = {}
for i in data_resample:
  keys = data_resample[i].keys()
  t = np.array([data_resample[i][j] for j in keys])
  data_tensor[f'{i}_tensor'] = tf.convert_to_tensor(t, dtype = tf.float32)

In [20]:
# Regrouping the shots, 'Other' and 'Rest' trials together in the 'data_tensor' dictionary
for n1,n2,n3 in zip(data_tensor, data_other, data_rest):
  data_tensor[n1] = tf.concat([data_tensor[n1],data_other[n2]], 0)
  data_tensor[n1] = tf.concat([data_tensor[n1],data_rest[n3]], 0)

In [60]:
# Splitting participants' tensor into data and labels
tensorvalues3 = list(data_tensor.values())
all_data3 = tf.concat(tensorvalues3, axis = 0)
X3 = all_data3[:,:,0:15].numpy()
y3 = all_data3[:,0,15].numpy()

In [64]:
# Train/test split
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size=0.2, random_state=42)

print("X_train:",X_train3.shape,
      "X_test:",X_test3.shape,
      "y_train:",y_train3.shape,
      "y_test:",y_test3.shape)

X_train: (2655, 400, 15) X_test: (664, 400, 15) y_train: (2655,) y_test: (664,)


In [53]:
save_pickle('/content/drive/MyDrive/Sam/File_saved/X_test3.pickle',X_test3)
save_pickle('/content/drive/MyDrive/Sam/File_saved/X_train3.pickle',X_train3)
#save_pickle('/content/drive/MyDrive/Sam/File_saved/X_valid3.pickle',X_valid3)
save_pickle('/content/drive/MyDrive/Sam/File_saved/y_test3.pickle',y_test3)
#save_pickle('/content/drive/MyDrive/Sam/File_saved/y_valid3.pickle',y_valid3)
save_pickle('/content/drive/MyDrive/Sam/File_saved/y_train3.pickle',y_train3)

<_io.BufferedWriter name='/content/drive/MyDrive/Sam/File_saved/y_train3.pickle'>